# ADU Deep Research System Testing Notebook

This notebook tests each component of the app-adu-deep-research-o system step by step.

## 1. Environment Setup & Import Check

In [ ]:
import os
import sys
import json
from pathlib import Path

# Add app directory to path
app_path = Path.cwd() / 'app'
if str(app_path) not in sys.path:
    sys.path.insert(0, str(app_path))

print(f"Working directory: {Path.cwd()}")
print(f"App path: {app_path}")
print(f"Python path includes: {sys.path[:3]}")

In [ ]:
# Check environment variables
env_vars = {
    "OPENAI_API_KEY": os.getenv("OPENAI_API_KEY", "NOT SET")[:20] + "...",
    "ORCHESTRATOR_MODEL": os.getenv("ORCHESTRATOR_MODEL", "gpt-5"),
    "SEARCH_MODEL": os.getenv("SEARCH_MODEL", "gpt-4o-mini-search-preview")
}

for key, value in env_vars.items():
    print(f"{key}: {value}")

if "NOT SET" in env_vars["OPENAI_API_KEY"]:
    print("\n⚠️ WARNING: OPENAI_API_KEY not set! Set it with:")
    print("export OPENAI_API_KEY='your-key-here'")

## 2. Test Module Imports

In [ ]:
# Test importing all modules
try:
    from app.config import get_settings
    print("✅ config module imported")
except Exception as e:
    print(f"❌ config module error: {e}")

try:
    from app.schemas import AnalyzeRequest, SearchBundle, AnalyzeResult
    print("✅ schemas module imported")
except Exception as e:
    print(f"❌ schemas module error: {e}")

try:
    from app.llm import responses_create
    print("✅ llm module imported")
except Exception as e:
    print(f"❌ llm module error: {e}")

try:
    from app.agents.prompts import SEARCH_SYSTEM, ORCH_SYSTEM
    print("✅ prompts module imported")
except Exception as e:
    print(f"❌ prompts module error: {e}")

try:
    from app.agents.searcher import search_everything
    print("✅ searcher module imported")
except Exception as e:
    print(f"❌ searcher module error: {e}")

try:
    from app.agents.orchestrator import run as orchestrate
    print("✅ orchestrator module imported")
except Exception as e:
    print(f"❌ orchestrator module error: {e}")

try:
    from app.tools.costs import compute_costs, CostInputs
    print("✅ costs tools imported")
except Exception as e:
    print(f"❌ costs tools error: {e}")

try:
    from app.tools.finance import estimate_rent, compute_finance
    print("✅ finance tools imported")
except Exception as e:
    print(f"❌ finance tools error: {e}")

## 3. Test Configuration

In [ ]:
# Test configuration loading
from app.config import get_settings

settings = get_settings()
print("Settings loaded:")
print(f"  - API Key: {'✅ Set' if settings.openai_api_key else '❌ Missing'}")
print(f"  - Base URL: {settings.openai_base_url or 'Default'}")
print(f"  - Orchestrator Model: {settings.orchestrator_model}")
print(f"  - Search Model: {settings.search_model}")

## 4. Test Cost Calculation Module (No API Required)

In [ ]:
from app.tools.costs import compute_costs, CostInputs

# Test basic cost calculation
test_inputs = CostInputs(
    adu_type="detached",
    size_sqft=750,
    hillside=False,
    vhfhsz=False,
    historic=False,
    sprinklers_required=False,
    separate_meter=True
)

print("Test Input:")
print(test_inputs.model_dump())
print("\nCalculated Costs:")
costs = compute_costs(test_inputs)
print(json.dumps(costs, indent=2))

In [ ]:
# Test with risk factors
risky_inputs = CostInputs(
    adu_type="detached",
    size_sqft=1000,
    hillside=True,
    vhfhsz=True,
    historic=False,
    sprinklers_required=True,
    separate_meter=True
)

print("Risky Property Input:")
print(risky_inputs.model_dump())
print("\nCalculated Costs (with risks):")
risky_costs = compute_costs(risky_inputs)
print(json.dumps(risky_costs, indent=2))

# Compare
print(f"\nCost increase due to risks: ${risky_costs['total_cost'] - costs['total_cost']:,.2f}")

## 5. Test Finance Module (No API Required)

In [ ]:
from app.tools.finance import estimate_rent, compute_finance

# Test rent estimation
test_cases = [
    ("94704", 450),  # Small unit
    ("94704", 750),  # Medium unit
    ("94704", 1000), # Large unit
]

print("Rent Estimates:")
for zipcode, size in test_cases:
    rent = estimate_rent(zipcode, size)
    print(f"  {size} sqft in {zipcode}: ${rent:,.2f}/month")

# Test finance calculations
print("\nFinancial Analysis:")
capex = costs['total_cost']  # From previous calculation
rent = estimate_rent("94704", 750)
finance = compute_finance(capex, rent)
print(json.dumps(finance, indent=2))

## 6. Test Searcher Agent (Requires API)

In [ ]:
# Only run if API key is set
if settings.openai_api_key:
    from app.agents.searcher import search_everything
    
    test_address = "690 Panoramic Way, Berkeley, CA"
    test_overrides = {
        "backyard_depth_ft": 32,
        "slope_degree": 18
    }
    
    print(f"Testing searcher for: {test_address}")
    print(f"User overrides: {test_overrides}")
    print("\nCalling search_everything()...")
    
    try:
        search_result = search_everything(test_address, test_overrides)
        print("\n✅ Search completed successfully!")
        print("\nSearch Results Structure:")
        print(f"  - Jurisdiction: {list(search_result.jurisdiction.keys())}")
        print(f"  - Zoning: {list(search_result.zoning.keys())}")
        print(f"  - Rules: {list(search_result.rules.keys())}")
        print(f"  - Risks: {list(search_result.risks.keys())}")
        print(f"  - Fees: {list(search_result.fees.keys())}")
        print(f"  - Citations: {len(search_result.citations)} sources")
        
        # Show sample data
        print("\nSample Jurisdiction Data:")
        print(json.dumps(search_result.jurisdiction, indent=2)[:500] + "...")
        
    except Exception as e:
        print(f"\n❌ Search failed: {e}")
        import traceback
        traceback.print_exc()
else:
    print("⚠️ Skipping API test - OPENAI_API_KEY not set")

## 7. Test Orchestrator Agent (Requires API)

In [ ]:
# Only run if API key is set and search was successful
if settings.openai_api_key and 'search_result' in locals():
    from app.agents.orchestrator import run as orchestrate
    
    test_targets = {
        "adu_type": "detached",
        "size_sqft": 750
    }
    
    print(f"Testing orchestrator for: {test_address}")
    print(f"Targets: {test_targets}")
    print("\nCalling orchestrate()...")
    
    try:
        result = orchestrate(test_address, test_targets, search_result)
        print("\n✅ Orchestration completed successfully!")
        
        print("\nResult Structure:")
        print(f"  - Search data: ✅")
        print(f"  - Cost breakdown: ✅")
        print(f"  - Finance analysis: ✅")
        print(f"  - Plan (Markdown): {len(result.plan_md)} characters")
        
        print("\nCost Summary:")
        print(f"  Total: ${result.costs.total_cost:,.2f}")
        
        print("\nFinance Summary:")
        print(f"  Monthly Rent: ${result.finance.est_rent_monthly:,.2f}")
        print(f"  ROI: {result.finance.roi_pct:.1f}%")
        print(f"  Payback: {result.finance.payback_years:.1f} years")
        
        print("\nGenerated Plan (first 500 chars):")
        print(result.plan_md[:500] + "...")
        
    except Exception as e:
        print(f"\n❌ Orchestration failed: {e}")
        import traceback
        traceback.print_exc()
else:
    print("⚠️ Skipping orchestrator test - prerequisites not met")

## 8. Test Full API Endpoint (End-to-End)

In [ ]:
# Test the complete flow through the API
if settings.openai_api_key:
    from app.schemas import AnalyzeRequest, AnalyzeTargets
    from app.main import analyze
    import asyncio
    
    # Create request
    request = AnalyzeRequest(
        address="690 Panoramic Way, Berkeley, CA",
        user_overrides={"backyard_depth_ft": 32, "slope_degree": 18},
        targets=AnalyzeTargets(adu_type="detached", size_sqft=750)
    )
    
    print("Test Request:")
    print(request.model_dump_json(indent=2))
    print("\nCalling API endpoint...")
    
    try:
        # Run async function
        result = await analyze(request)
        print("\n✅ API call successful!")
        
        # Parse result
        if hasattr(result, 'body'):
            data = json.loads(result.body)
        else:
            data = result
            
        print("\nAPI Response Summary:")
        print(f"  - Total Cost: ${data['costs']['total_cost']:,.2f}")
        print(f"  - ROI: {data['finance']['roi_pct']:.1f}%")
        print(f"  - Plan generated: {len(data['plan_md'])} characters")
        
    except Exception as e:
        print(f"\n❌ API call failed: {e}")
        import traceback
        traceback.print_exc()
else:
    print("⚠️ Skipping API test - OPENAI_API_KEY not set")

## 9. Test with Mock LLM Response (For Debugging)

In [ ]:
# Create a mock response for testing without API
from app.schemas import SearchBundle, Citation

# Mock search result
mock_search = SearchBundle(
    jurisdiction={
        "city": "Berkeley",
        "county": "Alameda",
        "state": "CA",
        "zipcode": "94704"
    },
    zoning={
        "code": "R-1",
        "adu_allowed": True,
        "max_adu_size": 1200
    },
    rules={
        "max_height_ft": 16,
        "setback_rear_ft": 4,
        "setback_side_ft": 4,
        "parking_required": False,
        "sprinklers_required": False,
        "separate_meter_suggested": True
    },
    risks={
        "hillside": True,
        "wildfire_vhfhsz": False,
        "historic_district": False,
        "fault_zone": True
    },
    fees={
        "permit_base": 3500,
        "impact_fee": 0,  # SB 13 exemption for <750 sqft
        "school_fee": 0,
        "sewer_connection": 2500
    },
    citations=[
        Citation(url="https://berkeley.gov/adu", title="Berkeley ADU Guidelines"),
        Citation(url="https://hcd.ca.gov/adu", title="California HCD ADU Handbook")
    ]
)

print("Mock Search Bundle created:")
print(f"  - Jurisdiction: {mock_search.jurisdiction['city']}, {mock_search.jurisdiction['state']}")
print(f"  - Zoning: {mock_search.zoning['code']} (ADU allowed: {mock_search.zoning['adu_allowed']})")
print(f"  - Risks identified: {sum(1 for v in mock_search.risks.values() if v)}")
print(f"  - Total fees: ${sum(mock_search.fees.values()):,.2f}")
print(f"  - Citations: {len(mock_search.citations)}")

In [ ]:
# Test orchestrator with mock data (no API needed for cost/finance calculations)
from app.tools.costs import compute_costs, CostInputs
from app.tools.finance import compute_finance, estimate_rent

# Calculate costs based on mock search data
cost_inputs = CostInputs(
    adu_type="detached",
    size_sqft=750,
    hillside=mock_search.risks.get("hillside", False),
    vhfhsz=mock_search.risks.get("wildfire_vhfhsz", False),
    historic=mock_search.risks.get("historic_district", False),
    sprinklers_required=mock_search.rules.get("sprinklers_required", False),
    separate_meter=mock_search.rules.get("separate_meter_suggested", False)
)

costs = compute_costs(cost_inputs)
rent = estimate_rent(mock_search.jurisdiction.get("zipcode"), 750)
finance = compute_finance(costs["total_cost"], rent)

print("Analysis Results (with mock data):")
print("\nCosts:")
print(json.dumps(costs, indent=2))
print("\nFinancials:")
print(json.dumps(finance, indent=2))

print("\n📊 Summary:")
print(f"  Total Investment: ${costs['total_cost']:,.2f}")
print(f"  Monthly Rent: ${finance['est_rent_monthly']:,.2f}")
print(f"  Annual NOI: ${finance['annual_net_income']:,.2f}")
print(f"  ROI: {finance['roi_pct']:.1f}%")
print(f"  Payback Period: {finance['payback_years']:.1f} years")

## 10. Performance & Error Testing

In [ ]:
# Test edge cases and error handling
from app.tools.costs import compute_costs, CostInputs

edge_cases = [
    # Minimum size
    CostInputs(adu_type="JADU", size_sqft=150),
    # Maximum complexity
    CostInputs(
        adu_type="detached",
        size_sqft=1200,
        hillside=True,
        vhfhsz=True,
        historic=True,
        sprinklers_required=True,
        separate_meter=True
    ),
    # Conversion type
    CostInputs(adu_type="conversion", size_sqft=400),
]

print("Edge Case Testing:")
for i, case in enumerate(edge_cases, 1):
    try:
        result = compute_costs(case)
        print(f"\nCase {i} ({case.adu_type}, {case.size_sqft} sqft):")
        print(f"  Total Cost: ${result['total_cost']:,.2f}")
        print(f"  Risk Costs: ${sum(result['risk_costs'].values()):,.2f}")
    except Exception as e:
        print(f"\nCase {i} FAILED: {e}")

## Summary & Next Steps

### Test Results:
- [ ] Environment setup
- [ ] Module imports
- [ ] Cost calculations
- [ ] Finance calculations
- [ ] Searcher agent (requires API)
- [ ] Orchestrator agent (requires API)
- [ ] End-to-end flow (requires API)

### Issues Found:
(Document any issues discovered during testing)

### Recommendations:
(Document recommendations for improvements)